In [ ]:
!pip install Quandl
!pip install investpy
!pip install missingno
!pip install pandas_ta

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import quandl
import investpy
import pandas_ta as ta

# Utils

In [ ]:
URL_array  = set()
def link2df(URL,col_name,join_df,join=True,check_column=True,check_URL = True,clear_URL_array=False,show_details=False):
    '''This function scraps the given link and returns dataframe
    __________
    Parameters:
        URL(string): URL to be scrapped from bitcoin website
        col_name(string): column name for dataframe
        join_df(variable)= dataframe withwhich output dataframe will be left joined on Date
        join(boolean)= iF True,join, else don't join
        check_column(boolean)= check if column name already exists
        check_URL(boolean)= check if URL is already processed
        clear_URL_array(boolean)= if true URL_processed array will be cleared
        show_details(boolean)= various details wil be printed such as scrapping first and last details, df head & df tail     
        '''
        
    print(f'processing {col_name}')

    #clear URL append array
    if clear_URL_array==True:
        URL_array.clear()

    #set join parameters if false
    if join == False:
        join_df = None
        check_column=False

    #process column name by making it lowercase and replacing spaces,commas, full stops
    col_name = col_name.lower().replace(',','').replace(" ", "_").replace(".", "_")

    #col_name validation if exists already
    if check_column==True and col_name in list(join_df.columns):
        print(f'column {col_name} already esists in dataframe, stopped here')
        return join_df

    #URL validation if processes already
    elif check_URL==True and URL in list(URL_array):
        print(f'{URL} is already processed, stopped here')
        return join_df 

    #web scrapping
    page = requests.get(URL)
    soup = page.content
    soup = str(soup)
    scraped_output = (soup.split('[[')[1]).split('{labels')[0][0:-2]
    if show_details == True:
        print('head')
        print({scraped_output[0:30]})
        print('tail')
        print({scraped_output[-30:]})

    processed_str = scraped_output.replace('new Date(','')
    processed_str = processed_str.replace(')','')
    processed_str = processed_str.replace('[','')
    processed_str = processed_str.replace(']','')
    processed_str = processed_str.replace('"','')

    processed_str_list = processed_str.split(',')
    date_list,data_list = processed_str_list[::2],processed_str_list[1::2]

    #validate column lengths
    if len(date_list)!=len(data_list):
        print(f'date & data length:{len(date_list),len(data_list),len(date_list)==len(data_list)}')

    #convert list data to a dataframe
    if join == False:
        df = pd.DataFrame()
        df['Date'] = pd.to_datetime(date_list)
        df[col_name] = data_list
        URL_array.add(URL)
        if show_details == True:
            print('*'*100)
            print('df head')
            print(df.head(1))
            print('*'*100)
            print('df tail')
            print(df.tail(1))
            print('*'*100)
            print(f'df shape{df.shape}')
            print('='*100)
            
        return df

    elif col_name not in list(join_df.columns) and join == True:
        df = pd.DataFrame()
        df['Date'] = pd.to_datetime(date_list)
        df[col_name] = data_list
        join_df = pd.merge(join_df,df,on=['Date'],how='left')
        URL_array.add(URL)
        if show_details == True:
            print('*'*100)
            print('df head')
            print(df.head(1))
            print('*'*100)
            print('df tail')
            print(df.tail(1))
            print('*'*100)
            print(f'output df shape= {df.shape},joined_df shape = {join_df.shape}')
            print('='*100)
            print(f'Number of duplicate columns in dataframe {df.columns.duplicated().sum()}')
            print('='*100)
    
        return join_df

# Web scrapping 

### Price

In [ ]:
final_df = investpy.get_crypto_historical_data(crypto='bitcoin',from_date='01/04/2013',to_date='19/04/2021')
final_df = final_df.reset_index()
final_df.drop(['Currency','Volume'],inplace=True,axis=1)
final_df.columns = ['Date','opening_price','highest_price','lowest_price','closing_price']
final_df

### Number of transactions in blockchain per day 

In [ ]:
final_df = link2df('https://bitinfocharts.com/comparison/bitcoin-transactions.html',
                   'transactions in blockchain',join_df=final_df,join=True)

### Average block size

In [ ]:
final_df = link2df('https://bitinfocharts.com/comparison/size-btc.html',
                   'avg block size',join_df=final_df,join=True)

### Number of unique (from) adresses per day

In [ ]:
final_df = link2df('https://bitinfocharts.com/comparison/sentbyaddress-btc.html',
                   'sent by adress',join_df=final_df,join=True)

### Average mining difficulty per day

In [ ]:
final_df = link2df('https://bitinfocharts.com/comparison/bitcoin-difficulty.html',
                   'avg mining difficulty',join_df=final_df,join=True)

### Average hashrate (hash/s) per day

In [ ]:
final_df = link2df('https://bitinfocharts.com/comparison/bitcoin-hashrate.html',
                   'avg hashrate',join_df=final_df,join=True)

### Mining Profitability USD/Day for 1 Hash/s

In [ ]:
final_df = link2df('https://bitinfocharts.com/comparison/bitcoin-mining_profitability.html',
                   'mining profitability',join_df=final_df,join=True)

### Sent coins in USD per day

In [ ]:
final_df = link2df('https://bitinfocharts.com/comparison/sentinusd-btc.html',
                   'Sent coins in USD',join_df=final_df,join=True)

### Average transaction fee, USD

In [ ]:
final_df = link2df('https://bitinfocharts.com/comparison/bitcoin-transactionfees.html',
                   'avg transaction fees',join_df=final_df,join=True)

### Median transaction fee, USD

In [ ]:
final_df = link2df('https://bitinfocharts.com/comparison/bitcoin-median_transaction_fee.html',
                   'median transaction fees',join_df=final_df,join=True)

### Average block time (minutes)

In [ ]:
final_df = link2df('https://bitinfocharts.com/comparison/bitcoin-confirmationtime.html',
                   'avg block time',join_df=final_df,join=True)

### Avg. Transaction Value, USD

In [ ]:
final_df = link2df('https://bitinfocharts.com/comparison/transactionvalue-btc.html',
                   'avg transaction value',join_df=final_df,join=True)

### Median Transaction Value, USD

In [ ]:
final_df = link2df('https://bitinfocharts.com/comparison/mediantransactionvalue-btc.html',
                   'median transaction value',join_df=final_df,join=True)

### Tweets per day

In [ ]:
final_df = link2df('https://bitinfocharts.com/comparison/tweets-btc.html',
                   'tweets',join_df=final_df,join=True)

### Google Trends to "Bitcoin" @ 2012-01-01

In [ ]:
final_df = link2df('https://bitinfocharts.com/comparison/google_trends-btc.html',
                   'google trends',join_df=final_df,join=True)

### Number of unique (from or to) addresses per day

In [ ]:
final_df = link2df('https://bitinfocharts.com/comparison/activeaddresses-btc.html',
                   'active addresses',join_df=final_df,join=True)

### Top 100 Richest Addresses to Total coins %

In [ ]:
final_df = link2df('https://bitinfocharts.com/comparison/top100cap-btc.html',
                   'top100 to total percentage',join_df=final_df,join=True)

### Average Fee Percentage in Total Block Reward

In [ ]:
final_df = link2df('https://bitinfocharts.com/comparison/fee_to_reward-btc.html',
                   'avg fee to reward',join_df=final_df,join=True)

### Total number of bitcoins in circulation

In [ ]:
btc_in_circulation_df = quandl.get("BCHAIN/TOTBC",authtoken='9ztFCcK4_e1xGo_gjzK7')
btc_in_circulation_df = btc_in_circulation_df.rename(columns={'Value': 'number_of_coins_in_circulation'})

### Bitcoin Miners Revenue

In [ ]:
miners_revenue_df = quandl.get("BCHAIN/MIREV",authtoken='9ztFCcK4_e1xGo_gjzK7')
miners_revenue_df = miners_revenue_df.rename(columns={'Value': 'miner_revenue'})

In [ ]:
#Filtering data as we are considering this peiod only
final_df = final_df[np.logical_and(final_df['Date'] >= '01/04/2013',final_df['Date'] <= '20/04/2020')].reset_index(drop=True)

final_df = pd.merge(final_df,btc_in_circulation_df,on=['Date'],how='left')
final_df = pd.merge(final_df,miners_revenue_df,on=['Date'],how='left')

# Change the index to Date
final_df = final_df.set_index("Date")

# Fill NaN values

In [ ]:
# replace null value to nan
final_df.replace(to_replace='null', value=np.nan,inplace=True)
final_df.drop(final_df.tail(1).index,inplace=True)

# Check NaN values
print(f"There is {final_df.isnull().values.sum()} NaN values")

In [ ]:
missing_values = pd.DataFrame(final_df.isna().sum(),columns=['missing_count'])
missing_values.sort_values(by='missing_count',ascending=False)

### Remove NaN values in tweets

In [ ]:
final_df['tweets'].fillna(final_df['tweets'].rolling(40, min_periods=1).mean()).bfill().astype(float).plot(x=final_df.index.values,y='tweets',figsize=(25,5),grid=True)
for i in list(final_df.loc[pd.isna(final_df['tweets']), :].index):
    plt.axvline(x=i,color='r',alpha=0.1)
plt.ylabel('n_tweets')
plt.title('Date vs n_tweets(with highlighted imputation)')
plt.show()
final_df['tweets'] = final_df['tweets'].fillna(final_df['tweets'].rolling(40, min_periods=1).mean()).bfill()

### Remove NaN values in active_addresses

In [ ]:
final_df['active_addresses'].fillna(final_df['active_addresses'].rolling(40, min_periods=1).mean()).bfill().astype(float).plot(x=final_df.index.values,y='active_addresses',figsize=(25,5),grid=True)
for i in list(final_df.loc[pd.isna(final_df['active_addresses']), :].index):
    plt.axvline(x=i,color='r',alpha=0.1)
plt.ylabel('active_addresses')
plt.title('Date vs active_addresses(with highlighted imputation)')
plt.show()
final_df['active_addresses'] = final_df['active_addresses'].fillna(final_df['active_addresses'].rolling(40, min_periods=1).mean()).bfill()

### Remove NaN values in top100_to_total_percentage

In [ ]:
final_df['top100_to_total_percentage'].fillna(final_df['top100_to_total_percentage'].rolling(40, min_periods=1).mean()).bfill().astype(float).plot(x=final_df.index.values,y='top100_to_total_percentage',figsize=(25,5),grid=True)
for i in list(final_df.loc[pd.isna(final_df['top100_to_total_percentage']), :].index):
    plt.axvline(x=i,color='r',alpha=0.1)
plt.ylabel('top100_to_total_percentage')
plt.title('Date vs top100_to_total_percentage(with highlighted imputation)')
plt.show()
final_df['top100_to_total_percentage'] = final_df['top100_to_total_percentage'].fillna(final_df['top100_to_total_percentage'].rolling(40, min_periods=1).mean()).bfill()

### Remove NaN values in avg_block_time

In [ ]:
final_df['avg_block_time'].fillna(final_df['avg_block_time'].rolling(40, min_periods=1).mean()).bfill().astype(float).plot(x=final_df.index.values,y='avg_block_time',figsize=(25,5),grid=True)
for i in list(final_df.loc[pd.isna(final_df['avg_block_time']), :].index):
    plt.axvline(x=i,color='r',alpha=0.1)
plt.ylabel('avg_block_time')
plt.title('Date vs avg_block_time(with highlighted imputation)')
plt.show()
final_df['avg_block_time'] = final_df['avg_block_time'].fillna(final_df['avg_block_time'].rolling(40, min_periods=1).mean()).bfill()

### Remarks

We fill the NaN values by using a rolling mean on 40 days. 

In [ ]:
missing_values = pd.DataFrame(final_df.isna().sum(),columns=['missing_count'])
missing_values.sort_values(by='missing_count',ascending=False)

In [ ]:
final_df['avg_hashrate'].fillna(final_df['avg_hashrate'].rolling(40, min_periods=1).mean()).bfill().astype(float).plot(x=final_df.index.values,y='avg_hashrate',figsize=(25,5),grid=True)


plt.ylabel('miner_revenue')
plt.title('Date vs miner_revenue(with highlighted imputation)')
plt.show()

# Features 

In [ ]:
final_df.dtypes

In [ ]:
final_df = final_df.astype("float")

In [ ]:
def feature_smoothening(df,feature_name,smoothening_type,smoothening_range=[7,30,90],show_plot=False,show_original_Feature_in_plot=True):
    if smoothening_type == 'sma':
        for j in smoothening_range:
            df[f'{smoothening_type}{j} {feature_name}'] = ta.sma(df[feature_name],j) 

    elif smoothening_type == 'var':
        for j in smoothening_range:
            df[f'{smoothening_type}{j} {feature_name}'] = ta.variance(df[feature_name],j)

    elif smoothening_type == 'stdev':
        for j in smoothening_range:
            df[f'{smoothening_type}{j} {feature_name}'] = ta.stdev(df[feature_name],j)
    
    elif smoothening_type == 'ema':
        for j in smoothening_range:
            df[f'{smoothening_type}{j} {feature_name}'] = ta.ema(df[feature_name],j)

    elif smoothening_type == 'wma':
        for j in smoothening_range:
            df[f'{smoothening_type}{j} {feature_name}'] = ta.wma(df[feature_name],j)

    elif smoothening_type == 'rsi':
        for j in smoothening_range:
             df[f'{smoothening_type}{j} {feature_name}'] = ta.rsi(df[feature_name],j)

    elif smoothening_type == 'roc':
        for j in smoothening_range:
            df[f'{smoothening_type}{j} {feature_name}'] = ta.roc(df[feature_name],j)  

    elif smoothening_type == 'dema':
        for j in smoothening_range:
            df[f'{smoothening_type}{j} {feature_name}'] = ta.dema(df[feature_name],j) 

    elif smoothening_type == 'tema':
        for j in smoothening_range:
            df[f'{smoothening_type}{j} {feature_name}'] = ta.tema(df[feature_name],j) 

    elif smoothening_type == 'bband_lower':
        for j in smoothening_range:
            bband_df = ta.bbands(df[feature_name],j)
            df[f'{smoothening_type}{j} {feature_name}'] = bband_df[f'BBL_{j}_2.0']

    elif smoothening_type == 'bband_upper':
        for j in smoothening_range:
            bband_df = ta.bbands(df[feature_name],j)
            df[f'{smoothening_type}{j} {feature_name}'] = bband_df[f'BBU_{j}_2.0']

    elif smoothening_type == 'macd':
        macd_df = ta.macd(df[feature_name])
        df[f'{smoothening_type} hist {feature_name}'] = macd_df['MACDh_12_26_9']
        df[f'{smoothening_type} signal {feature_name}'] = macd_df['MACDs_12_26_9']
        df[f'{smoothening_type} {feature_name}'] = macd_df['MACD_12_26_9']

    
    if show_plot == True and show_original_Feature_in_plot==True :
        df[[feature_name]+[i for i in list(df.columns) if i.split(" ")[-1] == feature_name and i.split(" ")[0][0:len(smoothening_type)] == smoothening_type]].plot(kind='line',figsize=(25,5))
        plt.grid()
        plt.title(f'Feature Smoothening-{feature_name} by {smoothening_type}')
        plt.xticks([])
        plt.show()

    elif show_plot == True and show_original_Feature_in_plot==False :
        df[[i for i in list(df.columns) if i.split(" ")[-1] == feature_name and i.split(" ")[0][0:len(smoothening_type)] == smoothening_type]].plot(kind='line',figsize=(25,5))
        plt.grid()
        plt.title(f'Feature Smoothening-{feature_name} by {smoothening_type}')
        plt.xticks([])
        plt.show()

In [ ]:
feature_list = [i for i in list(final_df.columns) if i not in ['Date','next_day_closing_price']]
for feature in feature_list:
    feature_smoothening(final_df,feature,'sma',show_plot=True)
    feature_smoothening(final_df,feature,'var',show_plot=True)
    feature_smoothening(final_df,feature,'stdev',show_plot=True)
    feature_smoothening(final_df,feature,'ema',show_plot=True)
    feature_smoothening(final_df,feature,'wma',show_plot=True)
    feature_smoothening(final_df,feature,'rsi',show_plot=True)
    feature_smoothening(final_df,feature,'roc',show_plot=True)
    feature_smoothening(final_df,feature,'dema',show_plot=True)
    feature_smoothening(final_df,feature,'tema',show_plot=True)
    feature_smoothening(final_df,feature,'bband_lower',show_plot=True)
    feature_smoothening(final_df,feature,'bband_upper',show_plot=True)
    feature_smoothening(final_df,feature,'macd',show_plot=True)

# Test of stationarity 

In [ ]:
# Load Statsmodels 
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

In [ ]:
# ADF Test
def adf_test(timeseries):
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)
# Call the function and run the test

adf_test(final_df["closing_price"])

The Augmented Dickey Test (ADF test)is one of the most common statistical tests used to test the stationarity of a given Time Series. It's a unit root test. 

Here, we observe the same order of magnitude for the ADF statistic -1.697262 with a p-value p = 0.43. The Time Series has then a  unit root and a trend.

# Data visualisation 

In [ ]:
final_df["closing_price"].plot()
plt.title("Bitcoin (BTC) prices from January 2012 to March 2021")
plt.xlabel("Date")
plt.ylabel("BTC price in USD")
plt.show()

In [ ]:
plt.title('Daily Lag')
pd.plotting.lag_plot(final_df["closing_price"], lag = 1)
plt.show()

plt.title('Monthly Lag')
pd.plotting.lag_plot(final_df["closing_price"], lag = 30)
plt.show()

plt.title('Yearly Lag')
pd.plotting.lag_plot(final_df["closing_price"], lag = 365)
plt.show()

Thanks to lag plots, one can observethe autocorrelation of the data. The data shows a correlation for daily lag plots. For monthly lag plots, it seems that there is no correlation around 2017 when the price of bitcoin skyrocketed.Concerning, the yearly lag plots, there is no correlation at all.

# Pre-processing

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# We create a column class as in the article 
# if P_btc[t+1]-P_btc[t] >=0 --> y[t] = +1 otherwise y[t]=0

final_df["class"] = (np.sign(final_df["closing_price"] - final_df["closing_price"].shift()) + 1)//2
final_df["class"][0] = 1 # otherwise, it's a NaN value

In [ ]:
data = final_df.drop("class",axis = 1)
Y = final_df["class"]

X_train, X_test, Y_train, Y_test = train_test_split(data, Y, test_size = 0.2)

# References 

### Scrapping 
https://github.com/rohansawant7978/bitcoin-price-forecasting

### Augmented Dickey-Fuller
https://www.analyticsvidhya.com/blog/2021/06/statistical-tests-to-check-stationarity-in-time-series-part-1/#




model ???

https://github.com/heliphix/btc_data

a classer


https://github.com/sergiovirahonda/AnomalyDetection/blob/main/Bitcoin/Notebook/forecasting-and-anomally-detection.ipynb


https://github.com/jaskirat111/Bitcoin-Time-Series-Forecast-ML-System


https://github.com/Pranjal-26/Time-series-Forecasting---TensorFlow/blob/main/Final_project%20(1).ipynb

https://github.com/Ashishkutchi/capstone-w2020-t2-g03-Time-Series-Forecasting-of-Bitcoin-Price

https://github.com/AmalVijayan/Time-Series-Forecasting---Predicting-Bitcoin-Prices

# TO DO 

### Scrap data used by the article 
Check if everything is scrapped

Remove NaN values
### Preprocess data 
linear interpolation to have missings cases

create train test data

### Visualisation 
pca 

graphs from the article

correlation between features

### Improvement 
Test on new datas --> avg block time --> must take more time 

# Remarks 

Generally, the missing values are the number of tweets --> difficult to approximate